### 5. Exercícios<a class="anchor" id="5"></a>

1. É possível o ES apresentar um valor menor (em módulo) que o VaR:   
a) sob o mesmo nível de confiança?   
b) sob um nível de confiança maior (ex: ES 95% e VaR 99%)?   
c) sob um nível de confiança menor (ex: ES 99% e VaR 95%)?

Resposta aqui

a) Não,visto que o ES é calculada através da média de todos os valores da série **abaixo** do VaR, ou seja, das perdas piores que o valor do VaR.

b) Sim. Nesse caso, visto que o ES possui um maior nível de confiança, o VaR tende a ter uma valor menor(em módulo) do que o VaR.

c) Não. Nesse caso, visto que o VaR possui um maior nível de confiança, o ES tende a ter uma valor ainda maior(em módulo) em relação ao VaR, do que no caso da letra a.

2. Calcule o VaR histórico com nível de confiança de 95% e HP=1, para a última data disponível de USD Spot (30/04/2021), utilizando uma amostra de 500 dias.

In [1]:
# Bibliotecas
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Resposta aqui 

# Importando o CSV

df_usd = pd.read_csv('USD.csv')

df_usd.head()

,paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim
0,1,1,"1,725","1,735",2010-01-04 09:23:00.0,Abertura
1,1,1,"1,728","1,73",2010-01-04 09:30:00.0,Intermediário
2,1,1,"1,727","1,729",2010-01-04 10:00:00.0,Intermediário
3,1,1,"1,728","1,73",2010-01-04 10:30:00.0,Intermediário
4,1,1,"1,728","1,73",2010-01-04 11:00:00.0,Intermediário


In [3]:
df_usd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18604 entries, 0 to 18603
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   paridadeCompra   18604 non-null  int64 
 1   paridadeVenda    18604 non-null  int64 
 2   cotacaoCompra    18604 non-null  object
 3   cotacaoVenda     18604 non-null  object
 4   dataHoraCotacao  18604 non-null  object
 5   tipoBoletim      18604 non-null  object
dtypes: int64(2), object(4)
memory usage: 872.2+ KB


In [4]:
# Convertendo os Valores das Colunas cotacaoCompra e cotacaoVenda para Float

df_usd['cotacaoCompra'] = df_usd['cotacaoCompra'].apply(lambda x: x.replace(',','.')).astype(float)
df_usd['cotacaoVenda'] = df_usd['cotacaoVenda'].apply(lambda x: x.replace(',','.')).astype(float)
df_usd.head()

,paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim
0,1,1,1.725,1.735,2010-01-04 09:23:00.0,Abertura
1,1,1,1.728,1.730,2010-01-04 09:30:00.0,Intermediário
2,1,1,1.727,1.729,2010-01-04 10:00:00.0,Intermediário
3,1,1,1.728,1.730,2010-01-04 10:30:00.0,Intermediário
4,1,1,1.728,1.730,2010-01-04 11:00:00.0,Intermediário


In [5]:
# Gerando as colunas retornoCompra, retornoVenda, PnLCompra, PnLVenda
# Multipliquei por 100 para que os resultados possam resultar em valores que não muito próximos de zero

df = df_usd
hp = 1
df['retornoCompra'] = (df['cotacaoCompra']/df['cotacaoCompra'].shift(hp) - 1)*100 # Retorno Compra
df['retornoVenda'] = (df['cotacaoVenda']/df['cotacaoVenda'].shift(hp) - 1)*100 # Retorno Venda
df['PnLCompra'] = df['retornoCompra'] - df['retornoCompra'].shift(hp - 1)*100 # Realizado Compra
df['PnLVenda'] = df['retornoVenda'] - df['retornoVenda'].shift(hp - 1)*100 # Realizado Venda

df = df.drop(['paridadeCompra', 'paridadeVenda', 'tipoBoletim'],axis=1)
df.sample(10).head()

,cotacaoCompra,cotacaoVenda,dataHoraCotacao,retornoCompra,retornoVenda,PnLCompra,PnLVenda
8400,1.9855,1.986,2013-03-04 10:06:33.749,0.060475,0.060459,-5.986998,-5.985490
5615,1.5800,1.590,2011-05-04 09:14:11.187,-0.516308,0.062933,51.114469,-6.230334
4085,1.6940,1.696,2010-12-21 13:00:21.417,-0.117925,-0.117786,11.674528,11.660777
1123,1.7710,1.773,2010-04-09 12:30:18.479,-0.112803,-0.112676,11.167513,11.154930
1048,1.7550,1.765,2010-04-05 09:18:25.854,-0.808229,-0.288119,80.014695,28.523812


### Cálculo do VaR

#### PNL Compra

In [6]:
amostra_PnLC2 = df.PnLCompra.iloc[len(df)-500:len(df)]
amostra_PnLC2

18104     4.734577
18105    -2.084211
18106    32.961482
18107    63.102118
18108    72.497343
           ...    
18599    10.886015
18600   -66.311547
18601   -57.246645
18602    -5.655266
18603    49.045102
Name: PnLCompra, Length: 500, dtype: float64

In [7]:
alpha = 0.05
var = amostra_PnLC2.quantile(alpha)
print("VaR 95% = R$ " + "{:.2f}".format(- var))

VaR 95% = R$ 88.75


#### PNL Venda

In [8]:
amostra_PnLV2 = df.PnLVenda.iloc[len(df)-500:len(df)]
amostra_PnLV2

18104     4.734033
18105    -2.083971
18106    32.957698
18107    63.094850
18108    72.488940
           ...    
18599    10.884797
18600   -66.304125
18601   -57.240280
18602    -5.654641
18603    49.039683
Name: PnLVenda, Length: 500, dtype: float64

In [9]:
alpha = 0.05
var = amostra_PnLV2.quantile(alpha)
print("VaR 95% = R$ " + "{:.2f}".format(- var))

VaR 95% = R$ 88.74


3. Calcule o ES histórico com nível de confiança de 97,5% e HP=1, para a última data disponível de USD Spot (30/04/2021), utilizando uma amostra de 1000 dias.

In [10]:
# Resposta aqui

#### PNL Compra

In [20]:
amostra_PnLC3 = df.PnLCompra.iloc[len(df)-1000:len(df)]
amostra_PnLC3

17604     5.188582
17605    -6.859936
17606   -27.976569
17607     9.791919
17608    25.891056
           ...    
18599    10.886015
18600   -66.311547
18601   -57.246645
18602    -5.655266
18603    49.045102
Name: PnLCompra, Length: 1000, dtype: float64

In [21]:
alpha = 0.25
var = amostra_PnLC3.quantile(alpha)
print("VaR 97,5% = R$ " + "{:.2f}".format(- var))

VaR 97,5% = R$ 28.47


#### PNL Venda

In [18]:
amostra_PnLV3 = df.PnLVenda.iloc[len(df)-1000:len(df)]
amostra_PnLV3

17604     5.187999
17605    -6.859165
17606   -27.973428
17607     9.790823
17608    25.888154
           ...    
18599    10.884797
18600   -66.304125
18601   -57.240280
18602    -5.654641
18603    49.039683
Name: PnLVenda, Length: 1000, dtype: float64

In [19]:
alpha = 0.25
var = amostra_PnLV3.quantile(alpha)
print("VaR 97,5% = R$ " + "{:.2f}".format(- var))

VaR 97,5% = R$ 28.72


4. Calcule a série temporal de VaR 99% e 97,5%, com HP=1, utilizando uma amostra de 1000 dias para todas as datas possíveis.   
Dica: Há 2844 dias com preços na base, removendo HP=1 para cálculo do primeiro retorno e 1000 para amostra do 1º VaR, teremos como resultado uma série de VaRs de tamanho 1843.

In [15]:
# Resposta aqui

In [16]:
# Gerando as colunas retornoCompra, retornoVenda, PnLCompra, PnLVenda
# Multipliquei por 100 para que os resultados possam resultar em valores que não muito próximos de zero

df = df_usd
hp = 1
df['retornoCompra'] = (df['cotacaoCompra']/df['cotacaoCompra'].shift(hp) - 1)*100 # Retorno Compra
df['retornoVenda'] = (df['cotacaoVenda']/df['cotacaoVenda'].shift(hp) - 1)*100 # Retorno Venda
df['PnLCompra'] = df['retornoCompra'] - df['retornoCompra'].shift(hp - 1)*100 # Realizado Compra
df['PnLVenda'] = df['retornoVenda'] - df['retornoVenda'].shift(hp - 1)*100 # Realizado Venda

df = df.drop(['paridadeCompra', 'paridadeVenda', 'tipoBoletim'],axis=1)
df.sample(10).head()

,cotacaoCompra,cotacaoVenda,dataHoraCotacao,retornoCompra,retornoVenda,PnLCompra,PnLVenda
17125,4.3752,4.3758,2020-02-19 12:04:17.459,0.070904,0.070894,-7.019510,-7.018547
3927,1.6810,1.6830,2010-12-08 10:30:15.748,-0.178147,-0.177936,17.636580,17.615658
3240,1.6830,1.6850,2010-10-07 15:30:17.306,-0.118694,-0.118554,11.750742,11.736811
13557,3.1458,3.1464,2017-04-12 12:03:20.4,0.111383,0.111362,-11.026955,-11.024850
9720,2.3415,2.3422,2014-03-20 11:11:22.312,-0.361702,-0.357356,35.808511,35.378201


In [24]:
# Criando uma função do cálculo do VaR para itera-lo

def calc_var(i, alpha):
    amostra_PnLC4 = df.PnLCompra.iloc[len(df)-1000:len(df)]
    var = amostra_PnLC4.quantile(alpha)
    return var

#### PNL Compra

In [29]:
alpha = 0.25
vars_c1 = []

for i in range(len(df)-1000):
    vars_c1.append(calc_var(i, alpha))
print(vars_c1)

[-28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.46733114

In [26]:
alpha = 0.01
vars_c2 = []

for i in range(len(df)-1000):
    vars_c2.append(calc_var(i, alpha))
print(vars_c2)

[-168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.8343519

In [31]:
df_vars_c = pd.DataFrame()

df_vars_c['vars_c1'] = vars_c1
df_vars_c['vars_c2'] = vars_c2

df_vars_c.head()

,vars_c1,vars_c2
0,-28.467331,-168.834352
1,-28.467331,-168.834352
2,-28.467331,-168.834352
3,-28.467331,-168.834352
4,-28.467331,-168.834352


In [32]:
alpha = 0.25
vars_v1 = []

for i in range(len(df)-1000):
    vars_v1.append(calc_var(i, alpha))
print(vars_v1)

[-28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.467331142098136, -28.46733114

In [33]:
alpha = 0.01
vars_v2 = []

for i in range(len(df)-1000):
    vars_v2.append(calc_var(i, alpha))
print(vars_v2)

[-168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.83435196318726, -168.8343519

In [34]:
df_vars_v = pd.DataFrame()

df_vars_v['vars_v1'] = vars_v1
df_vars_v['vars_v2'] = vars_v2

df_vars_v.head()

,vars_v1,vars_v2
0,-28.467331,-168.834352
1,-28.467331,-168.834352
2,-28.467331,-168.834352
3,-28.467331,-168.834352
4,-28.467331,-168.834352
